# Chapter 2 - Patterns for Cleaner Python
___

## 2.1 Assertions

### SUMMARY

- Python’s assert statement is a debugging aid that tests a condition as an internal self-check in your program.
- Asserts should only be used to **help developers identify bugs**.
  They’re not a mechanism for handling run-time errors.
- Asserts can be globally disabled with an interpreter setting. Hence they are vulnerable to security breaches. 

### What are they?

- Internal-self checks for your program
- They keep track of impossible conditions in your code

### Syntax

```python
assert_stmt ::= "assert" expression1 ["," expression2]

# At run time it gets converted to:
if __debug__:
    if not expression1:
        raise AssertionError(expression2)

```

### Example

```python
assert 0 <= price <= product['price']
```

### Reasons for using assertions

- Speed up debugging efforts
- Makes code more maintainable
- exception stacktrace points out the _exact line of code containing the failed assertion_

### What not to use them for?

- You shouldn't use them as a way to handle _run-time errors_
- _Don't use them for data validation_ as they can be globally disabled. Use  conditional logic and raise validation exceptions
- They can introduce security risks and bugs into your applications if you are not careful

### Common pitfalls

1. Introducing security risks

```python
# if assertions are disabled the security checks will not take place
# also the product id may be invalid which opens the door to denial of service attacks

#### INCORRECT WAY ####
def delete_product(prod_id, user):
    assert user.is_admin(), 'Must be admin'
    assert store.has_product(prod_id), 'Unknown product'
    store.get_product(prod_id).delete()


#### CORRECT WAY ####
def delete_product(product_id, user):
    if not user.is_admin():
        raise AuthError('Must be admin to delete')
    if not store.has_product(product_id):
        raise ValueError('Unknown product id')
    store.get_product(product_id).delete()

```

2. Asserts that never fail

It's suprisingly easy to write Python assert statements that always evaluate to true. This has to do with non-empty tuples always being truthy in Python. If you pass a tuple to an assert statement, it leads to the assert condition always being true.

```python

#### EXAMPLE OF ASSERTIONS THAT ALWAYS EVALUATES TO TRUE ####

assert(1 == 2, 'This should fail')


assert (
    counter == 10,
    'It should have counted all the items'
)
```


In [7]:
# example assertion use
def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price'], "invalid price"
    return price


# price in cents
shoes = {
    'name': 'Fancy Shoes',
    'price': 14900
}

In [8]:
# valid input
print(apply_discount(shoes, 0.25))

11175


In [9]:
# invalid input
print(apply_discount(shoes, 2))

AssertionError: invalid price

___
## 2.2 Context Managers

In [10]:
# read with context handler
# context managers abstractify management of files 
# they also can account for an exception unlike the try finally structure
with open(data_dir/'lorem_ipsum.txt') as file:
    print(file.read())

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.


In [11]:
# we can add this type of context management to our own classes
class ManagedFile:
    def __init__(self,name):
        self.name = name
    
    def __enter__(self):
        print('opening file')
        self.file = open(self.name,'r')
        return self.file
    
    def __exit__(self,exc_type,exc_val,exc_tb):
        print('closing file')
        if self.file:
            self.file.close()

In [12]:
with ManagedFile(data_dir/'lorem_ipsum.txt') as f:
    print(f.read())

opening file
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
closing file


In [14]:
# here is another way to do this with decorators 
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        print('opening file')
        f = open(name, 'r')
        yield f
    finally:
        print('closing file')
        f.close()


with managed_file(data_dir/'lorem_ipsum.txt') as f:
    print(f.read())


opening file
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
closing file


In [15]:
# alternative using try finally 
# this will fail to account for exceptions

f = open(data_dir/'lorem_ipsum.txt')
try:
    print(f.read())
finally:
    f.close()

Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.


___
## 2.3 Underscores and their uses

### Uses with classes

In [16]:
# this will set a Test class variable
_Test__mangled = 23

class Test:
    def __init__(self):
        self.foo = 21
        self._bar = 17
        self.__set_car(13)
    
    def __set_car(self,value):
        if value <= 0:
            raise ValueError('Invalid value')
        else:
            self.__car = value
    
    def get_car(self):
        return self.__car
    
    def mangled_test(self):
        return __mangled

In [17]:
t = Test()

In [18]:
t.foo

21

In [19]:
t._bar

17

In [20]:
t.get_car()

13

In [21]:
t.mangled_test()

23

In [32]:
[x for x in dir(t) if 'Test' in x]

['_Test__car', '_Test__set_car']

### Placeholders

In [33]:
for _ in range(5):
    print('hello')

hello
hello
hello
hello
hello


### Import caveats

```python
# my_module.py:
def external_func():
    return 23
def _internal_func():
    return 42  
```

```python
# main.py 
from my_module import * 
external_func() 
_internal_func() # this will not work 
```

```python 
# main.py
import my_module 
my_module.external_func() 
my_module._internal_func() # this will work
```


### Name Mangling and Inheritance 

In [34]:
class Test: 
    def __init__(self):
        self.foo = 11
        self._bar = 23 
        self.__baz = 42
        
class ExtendedTest(Test):
    def __init__(self):
        super().__init__()
        self.foo = 'overidden'
        self._bar = 'overidden'
        self.__baz = 'overidden'
        
    def __set_baz(self,value):
        if value not in ['red', 'blue', 'green']:
            raise ValueError('Please enter red, blue or green')
        self.__baz = value
    
    def get_baz(self):
        return self.__baz

In [53]:
t = ExtendedTest()

In [54]:
dir(t)

['_ExtendedTest__baz',
 '_ExtendedTest__set_baz',
 '_Test__baz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo',
 'get_baz']

In [55]:
# overidden as expected 
t.foo

'overidden'

In [56]:
# overidden as expected
t._bar

'overidden'

In [57]:
# name mangling for parent class
t._Test__baz

42

In [58]:
# name mangling for child class
t._ExtendedTest__baz

'overidden'

In [59]:
# this will access the pseudo-private set_baz method
t._ExtendedTest__set_baz('red')

In [60]:
# we can see this class instance has baz set to red
t.get_baz()

'red'

In [61]:
# and the mangled parent variable is untouced
t._Test__baz

42

___
## 2.5 String Formatting

In [62]:
num = 50159747054
name = 'Bob'

### Old Style 

In [63]:
# x placeholder in string converts number to hexadecimal
'Hello, %x' % num

'Hello, badc0ffee'

In [64]:
'Hey %(name)s, there is a 0x%(num)x error!'% {
    'name' : name, 'num' : num
}

'Hey Bob, there is a 0xbadc0ffee error!'

### New Style

In [65]:
'Hello, {}'.format(name)

'Hello, Bob'

In [66]:
'Hey {name}, there is a 0x{num:x} error!'.format(name=name, num=num)

'Hey Bob, there is a 0xbadc0ffee error!'

### Literal String Interpolation

Reference: https://zetcode.com/python/fstring/

In [67]:
f'Hello, {name}!'

'Hello, Bob!'

In [69]:
# evaluate expression inline
a = 5
b = 10
f'Five plus ten is {a + b} and not {2 * (a + b)}.'

'Five plus ten is 15 and not 30.'

In [70]:
f"Hey {name}, there's a {num:.2e} error!"

"Hey Bob, there's a 5.02e+10 error!"

### Template Strings

In [71]:
from string import Template

t = Template('Hey, $name!')
t.substitute(name = name)

'Hey, Bob!'

In [72]:
templ_string = 'Hey $name, there is a $num error!'
Template(templ_string).substitute(name=name, num=hex(num))

'Hey Bob, there is a 0xbadc0ffee error!'

### Attack vector of format string

Highlights the important use case of Template strings

In [73]:
# SECRET CAN BE ACCESSED WITH FORMAT STRING
SECRET = 'this-is-a-secret'

class Error:
    def __init__(self):
        pass

# user can trigger error and pass input    
err = Error()
user_input = '{error.__init__.__globals__[SECRET]}'
# Uh-oh...
user_input.format(error=err)

'this-is-a-secret'

In [74]:
# the template string closes the attack vector
# ie. you cannot use the error 
user_input = '${error.__init__.__globals__[SECRET]}'
Template(user_input).substitute(error=err)

ValueError: Invalid placeholder in string: line 1, col 1